# Some Boilerplate stuff

In [1]:
# You may prefer to upload the data to your google drive and mount your google drive to this colab, 
# because the data will be erased if you stop using this colab for a while.
# Uncomment the code below to do so. After mounting, navigate to the appropriate folder, right click, and "copy path".
# Assign DATA_DIR global variable to that path.
# For more mounting instructions: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# If imported from google drive, config for your file directory. Mine is 'lm_data'.
DATA_DIR = "./drive/MyDrive/nlp-final-project/data"

# the goal is that DATA_DIR points to where the training/validation/test data is. 

In [3]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 3.8 MB 37.4 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 17.2 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 134 kB 39.9 MB/s 
     |████████████████████████████████| 212 kB 31.3 MB/s 
     |████████████████████████████████| 127 kB 45.9 MB/s 
     |████████████████████████████████| 6.5 MB 40.6 MB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
     |████████████████████████████████| 271 kB 37.1 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 144 kB 50.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=76ba7f32c3c4c86c980bc41717a78ecb71bf75c4879e650b97d0f7fff60965cb
  Stored in directory: /root/.cache/pip/whe

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,918 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [6]:
import transformers

print(transformers.__version__)

4.17.0


In [7]:
model_checkpoint = "bert-base-uncased"

batch_size = 16

# Loading Data

In [8]:
import csv
import ast
from datasets import Dataset, load_metric, DatasetDict

# Get raw datasets
'''
1.1 VUA
get raw dataset as a list:
  Each element is a triple:
    a sentence: string
    a list of labels: 
    a list of pos: 
'''
pos_set = set()
raw_train_vua = {}
raw_train_vua["tokens"] = []
raw_train_vua["tags"] = []
raw_train_vua["pos"] = []
with open(f'{DATA_DIR}/VUAsequence/VUA_seq_formatted_train.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_train_vua["tokens"].append(line[2].split())
        raw_train_vua["tags"].append(label_seq)
        raw_train_vua["pos"].append(pos_seq)
        pos_set.update(pos_seq)

raw_val_vua = {}
raw_val_vua["tokens"] = []
raw_val_vua["tags"] = []
raw_val_vua["pos"] = []
with open(f'{DATA_DIR}/VUAsequence/VUA_seq_formatted_val.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_val_vua["tokens"].append(line[2].split())
        raw_val_vua["tags"].append(label_seq)
        raw_val_vua["pos"].append(pos_seq)
        pos_set.update(pos_seq)

raw_test_vua = {}
raw_test_vua["tokens"] = []
raw_test_vua["tags"] = []
raw_test_vua["pos"] = []
with open(f'{DATA_DIR}/VUAsequence/VUA_seq_formatted_test.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_test_vua["tokens"].append(line[2].split())
        raw_test_vua["tags"].append(label_seq)
        raw_test_vua["pos"].append(pos_seq)
        pos_set.update(pos_seq)

dataset_dict = {}
dataset_dict["train"] = Dataset.from_dict(raw_train_vua)
dataset_dict["test"] = Dataset.from_dict(raw_test_vua)
dataset_dict["validation"] = Dataset.from_dict(raw_val_vua)

datasets = DatasetDict(dataset_dict)

In [9]:
from transformers import AutoTokenizer

    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [10]:
label_all_tokens = True

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    pos = examples["pos"]
    labels = []
    pos_x = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        pos_element = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
                pos_element.append("none")
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
                pos_element.append(pos[i][word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                pos_element.append(pos[i][word_idx] if label_all_tokens else "none")
            previous_word_idx = word_idx
        labels.append(label_ids)
        pos_x.append(pos_element)


    tokenized_inputs["labels"] = labels
    tokenized_inputs["pos_x"] = pos_x
    return tokenized_inputs

In [12]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'pos', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'pos_x'],
        num_rows: 6323
    })
    test: Dataset({
        features: ['tokens', 'tags', 'pos', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'pos_x'],
        num_rows: 2694
    })
    validation: Dataset({
        features: ['tokens', 'tags', 'pos', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'pos_x'],
        num_rows: 1550
    })
})


# Train model

In [13]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [14]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"bert-lets-go",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
)

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
metric = load_metric("seqeval")

In [17]:
import numpy as np
label_list = ["O", "I"]
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos, pos_x, tags. If tokens, pos, pos_x, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6323
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3960


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.164151,0.713319,0.463716,0.562052,0.933755
2,0.180900,0.148877,0.694598,0.622045,0.656323,0.943262
3,0.097300,0.169409,0.705617,0.645684,0.674322,0.945338
4,0.055400,0.186674,0.678444,0.680869,0.679654,0.943828
5,0.055400,0.232079,0.670807,0.682793,0.676747,0.943380
6,0.028900,0.265130,0.683799,0.672897,0.678304,0.944206
7,0.015500,0.301915,0.702491,0.666575,0.684062,0.945503
8,0.009300,0.316019,0.690362,0.675371,0.682784,0.944796
9,0.006900,0.340407,0.706210,0.662727,0.683778,0.945503
10,0.006900,0.346569,0.705226,0.660253,0.681999,0.945456


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos, pos_x, tags. If tokens, pos, pos_x, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
Saving model checkpoint to bert-lets-go/checkpoint-500
Configuration saved in bert-lets-go/checkpoint-500/config.json
Model weights saved in bert-lets-go/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-lets-go/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-lets-go/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos, pos_x, tags. If tokens, pos, pos_x, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this mes

TrainOutput(global_step=3960, training_loss=0.05035549625001772, metrics={'train_runtime': 1660.1876, 'train_samples_per_second': 38.086, 'train_steps_per_second': 2.385, 'total_flos': 1776155517698856.0, 'train_loss': 0.05035549625001772, 'epoch': 10.0})

In [34]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: pos, tokens, tags, pos_x. If pos, tokens, tags, pos_x are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16


{'epoch': 10.0,
 'eval_accuracy': 0.9466830234972162,
 'eval_f1': 0.6853980417198808,
 'eval_loss': 0.40544694662094116,
 'eval_precision': 0.7084188911704312,
 'eval_recall': 0.6638262781748213,
 'eval_runtime': 15.2434,
 'eval_samples_per_second': 101.683,
 'eval_steps_per_second': 6.363}

# Evaluate model

In [20]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

label_list = ["O", "I"]
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos, pos_x, tags. If tokens, pos, pos_x, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2694
  Batch size = 16


{'_': {'precision': 0.7123137598597721, 'recall': 0.6395829234703915, 'f1': 0.673991914584845, 'number': 5083}, 'overall_precision': 0.7123137598597721, 'overall_recall': 0.6395829234703915, 'overall_f1': 0.673991914584845, 'overall_accuracy': 0.9440978376462248}


## Evaluate for individual POS

In [21]:
pos = tokenized_datasets["test"]["pos_x"]
pos_groups = {}
pos_groups['NOUN'] = {}
pos_groups['NOUN']["label"] = []
pos_groups['NOUN']["pred"] = []
pos_groups['VERB'] = {}
pos_groups['VERB']["label"] = []
pos_groups['VERB']["pred"] = []
pos_groups['ADJ'] = {}
pos_groups['ADJ']["label"] = []
pos_groups['ADJ']["pred"] = []
pos_groups['ADV'] = {}
pos_groups['ADV']["label"] = []
pos_groups['ADV']["pred"] = []
pos_groups['PART'] = {}
pos_groups['PART']["label"] = []
pos_groups['PART']["pred"] = []

for i in range(len(true_predictions)):

    for j in range(len(true_predictions[i])):
        if(pos[i][j] in pos_groups):
            pos_groups[pos[i][j]]["label"].append(true_labels[i][j])
            pos_groups[pos[i][j]]["pred"].append(true_predictions[i][j])
print("NOUN")
results = metric.compute(predictions=[pos_groups['NOUN']["pred"]], references=[pos_groups['NOUN']["label"]])
print(results)
print("VERB")
results = metric.compute(predictions=[pos_groups['VERB']["pred"]], references=[pos_groups['VERB']["label"]])
print(results)
print("ADJ")
results = metric.compute(predictions=[pos_groups['ADJ']["pred"]], references=[pos_groups['ADJ']["label"]])
print(results)
print("ADP")
results = metric.compute(predictions=[pos_groups['ADV']["pred"]], references=[pos_groups['ADV']["label"]])
print(results)
print("PART")
results = metric.compute(predictions=[pos_groups['PART']["pred"]], references=[pos_groups['PART']["label"]])
print(results)

NOUN
{'_': {'precision': 0.7711670480549199, 'recall': 0.7064989517819706, 'f1': 0.737417943107221, 'number': 954}, 'overall_precision': 0.7711670480549199, 'overall_recall': 0.7064989517819706, 'overall_f1': 0.737417943107221, 'overall_accuracy': 0.9540056550424129}
VERB
{'_': {'precision': 0.7054125998225377, 'recall': 0.6537828947368421, 'f1': 0.678617157490397, 'number': 1216}, 'overall_precision': 0.7054125998225377, 'overall_recall': 0.6537828947368421, 'overall_f1': 0.678617157490397, 'overall_accuracy': 0.9394242121514849}
ADJ
{'_': {'precision': 0.6236786469344608, 'recall': 0.5230496453900709, 'f1': 0.5689488910318226, 'number': 564}, 'overall_precision': 0.6236786469344608, 'overall_recall': 0.5230496453900709, 'overall_f1': 0.5689488910318226, 'overall_accuracy': 0.9080841638981174}
ADP
{'_': {'precision': 0.6304909560723514, 'recall': 0.6054590570719603, 'f1': 0.6177215189873418, 'number': 403}, 'overall_precision': 0.6304909560723514, 'overall_recall': 0.6054590570719603,

## Print error analysis

In [22]:
print(predictions)
print(labels)

result = []
words = []
for ids in tokenized_datasets["test"]["input_ids"]:
    words.append(tokenizer.convert_ids_to_tokens(ids))
for i in range(len(true_labels)):
    result_entry_list = []
    for j in range(len(true_predictions[i])):
        result_entry_list.append(f"{words[i][j+1]}({true_predictions[i][j]} {true_labels[i][j]})")
    result_entry = " ".join(result_entry_list)
    result_entry = f"{result_entry}\n\n"
    result.append(result_entry)

f = open(f'{DATA_DIR}/predictions/vua_seq_test_predictions_BERTsequence_vua.txt', 'w+')
f.writelines(result)
f.close()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[-100    0    0 ... -100 -100 -100]
 [-100    0    0 ... -100 -100 -100]
 [-100    1    0 ... -100 -100 -100]
 ...
 [-100    0    0 ... -100 -100 -100]
 [-100    0    1 ... -100 -100 -100]
 [-100    0    0 ... -100 -100 -100]]


# Retokenization

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

label_list = ["O", "I"]
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [38]:
def detokenize(examples, predictions, labels):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    tokens = examples["tokens"]
    re_preds = []
    re_labels = []
    for i in range(len(examples[f"tags"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        re_pred = ["O"] * len(tokens[i])
        re_label = ["O"] * len(tokens[i])
        for j, word_idx in enumerate(word_ids):
            if j >= len(labels[i]):
                break
            if word_idx is None:
                continue
            else:
                if labels[i][j] == 'I':
                    re_label[word_idx] = "I"
                if predictions[i][j] == 'I':
                    re_pred[word_idx] = "I"
        re_labels.append(re_label)
        re_preds.append(re_pred)
    return (re_preds, re_labels)

In [39]:
re_preds, re_labels = detokenize(datasets["test"], true_predictions, true_labels)

results = metric.compute(predictions=re_preds, references=re_labels)
print(results)

{'_': {'precision': 0.7232574189095928, 'recall': 0.6421568627450981, 'f1': 0.6802986043492372, 'number': 4896}, 'overall_precision': 0.7232574189095928, 'overall_recall': 0.6421568627450981, 'overall_f1': 0.6802986043492372, 'overall_accuracy': 0.9441554559043348}
